# Setup

In [45]:
import pandas as pd
import regex
import numpy as np
import requests
import pprint
import time
import json
import re, string
import datetime

In [48]:
print('Version check:')
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('regex: {}'.format(regex.__version__))
print('requests: {}'.format(requests.__version__))
print('re: {}'.format(re.__version__))

Version check:
pandas: 1.0.3
numpy: 1.18.3
regex: 2.5.78
requests: 2.23.0
re: 2.2.1


# API Scraping

In [5]:
example = "https://liquipedia.net/starcraft2/api.php?action=askargs&format=json&conditions=Has_tournament_name::2016%20Global%20StarCraft%20II%20League%20Season%201/Code%20S%7CHas_player_left_score::%2B%7CHas_player_right_score::%2B&printouts=Has_player_left%7CHas_player_right%7CHas_player_left_race%7CHas_player_right_race%7CHas_player_left_score%7CHas_player_right_score&7CHas_montth_and_day&parameters=limit%3D100%7Coffset%3D0"

In [6]:
base_url = 'https://liquipedia.net/starcraft2/api.php'
test_page = '2016_Global_StarCraft_II_League_Season_1/Code_S'

headers = {
    'User-Agent': 'GSL_data_scraper_TerranCraft',
    'From': 'bxiao1816@gmail.com'
}

In [7]:
response = requests.get(example, headers=headers)

In [10]:
obj = json.loads(response.text)
obj_formatted = json.dumps(obj)
print(obj_formatted)

eague Season 1/Code S#Match Solar vs SpeeD at March 4, 2016 - 18:30 in Match 1": {"printouts": {"Has_player_left": ["Solar"], "Has_player_right": ["SpeeD"], "Has_player_left_race": ["z"], "Has_player_right_race": ["t"], "Has_player_left_score": [2], "Has_player_right_score": [1]}, "fulltext": "2016 Global StarCraft II League Season 1/Code S#Match Solar vs SpeeD at March 4, 2016 - 18:30 in Match 1", "fullurl": "https://liquipedia.net/starcraft2/2016_Global_StarCraft_II_League_Season_1/Code_S#Match_Solar_vs_SpeeD_at_March_4.2C_2016_-_18:30_in_Match_1", "namespace": 0, "exists": "1", "displaytitle": "2016 Global StarCraft II League Season 1: Code S"}, "2016 Global StarCraft II League Season 1/Code S#Match Solar vs soO at April 2, 2016 - 18:30 in Match 1": {"printouts": {"Has_player_left": ["Solar"], "Has_player_right": ["soO"], "Has_player_left_race": ["z"], "Has_player_right_race": ["z"], "Has_player_left_score": [0], "Has_player_right_score": [2]}, "fulltext": "2016 Global StarCraft II 

In [ ]:
# can process based on date

In [13]:
type(obj)

dict

In [16]:
obj.keys()

dict_keys(['warnings', 'query'])

In [18]:
obj['query'].keys()

dict_keys(['printrequests', 'results', 'serializer', 'version', 'meta'])

In [34]:
key1 = list(obj['query']['results'].keys())[0]
key1

'2016 Global StarCraft II League Season 1/Code S#Match Bunny vs MyuNgSiK at April 8, 2016 - 18:30 in Match 4'

In [90]:
len(obj['query']['results'].keys())

67

In [44]:
# Find match string
match_pattern = 'Match \d'
match_re = re.compile(match_pattern)
test_match = match_re.search(key1)
test_match.group()[-1] # grab match number

'4'

In [74]:
(\s\d{1,2},\s\d{4})

SyntaxError: unexpected character after line continuation character (<ipython-input-74-13b71ead9389>, line 1)

In [79]:
date_match = '(January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}'
date_re = re.compile(date_match)
date_re_match = date_re.search(key1)
date_re_match.group()

'April 8, 2016'

In [87]:
date_re_match.group()[-4:]

'2016'

In [43]:
year_match = '[0-9]{4}'
year_re = re.compile(year_match)
year_re_match = year_re.search(key1)
year_re_match.group()

'2016'

In [97]:
season_match = 'Season\s\d'
season_re = re.compile(season_match)
season_re_match = season_re.search(key1)
season_re_match.group()[-1]

'1'

In [35]:
obj['query']['results'][key1]

{'printouts': {'Has_player_left': ['Bunny'],
  'Has_player_right': ['MyuNgSiK'],
  'Has_player_left_race': ['t'],
  'Has_player_right_race': ['p'],
  'Has_player_left_score': [1],
  'Has_player_right_score': [2]},
 'fulltext': '2016 Global StarCraft II League Season 1/Code S#Match Bunny vs MyuNgSiK at April 8, 2016 - 18:30 in Match 4',
 'fullurl': 'https://liquipedia.net/starcraft2/2016_Global_StarCraft_II_League_Season_1/Code_S#Match_Bunny_vs_MyuNgSiK_at_April_8.2C_2016_-_18:30_in_Match_4',
 'namespace': 0,
 'exists': '1',
 'displaytitle': '2016 Global StarCraft II League Season 1: Code S'}

In [83]:
obj['query']['results'][key1]['printouts']

{'Has_player_left': ['Bunny'],
 'Has_player_right': ['MyuNgSiK'],
 'Has_player_left_race': ['t'],
 'Has_player_right_race': ['p'],
 'Has_player_left_score': [1],
 'Has_player_right_score': [2]}

In [82]:
obj['query']['results'][key1]['printouts']['Has_player_left'][0]

'Bunny'

In [ ]:
# 2D dataframe
# sort by date
# check total matches played for group # or playoffs

In [84]:
gsl_results = list()

In [88]:
def get_date(key_string):
    date_match = '(January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2},\s\d{4}'
    date_re = re.compile(date_match)
    date_re_match = date_re.search(key_string)

    date = date_re_match.group()
    year = date[-4:]

    return date, year

In [89]:
def get_match_no(key_string):
    match_pattern = 'Match \d'
    match_re = re.compile(match_pattern)
    test_match = match_re.search(key_string)
    match_no = test_match.group()[-1] # grab match number

    return match_no

In [98]:
def get_season(key_string):
    season_match = 'Season\s\d'
    season_re = re.compile(season_match)
    season_re_match = season_re.search(key_string)
    season = season_re_match.group()[-1]

    return season

In [111]:
obj['query']['results'][key1]['printouts']['Has_player_left'][0]

'Bunny'

In [114]:
obj['query']['results'][key1]['printouts']['Has_player_right'][0]

'MyuNgSiK'

In [142]:
def parse_match(input_data, match):
    data = dict()
    data['player_left'] = input_data['query']['results'][match]['printouts']['Has_player_left'][0]
    data['player_left_race'] = input_data['query']['results'][match]['printouts']['Has_player_left_race'][0]
    data['player_right'] = input_data['query']['results'][match]['printouts']['Has_player_right'][0]
    data['player_right_race'] = input_data['query']['results'][match]['printouts']['Has_player_right_race'][0]
    # grab score
    score_left = input_data['query']['results'][match]['printouts']['Has_player_left_score'][0]
    score_right = input_data['query']['results'][match]['printouts']['Has_player_right_score'][0]
    # input score
    score = str(score_left) + '-' + str(score_right)
    #print(score)
    data['score'] = score

    if int(score_left) > int(score_right):
        data['winner'] = data['player_left']
        data['loser'] = data['player_right']
    else:
        data['winner'] = data['player_right']
        data['loser'] = data['player_left'] 

    data['match_no'] = get_match_no(match)
    date, year = get_date(match)
    data['match_date'] = date
    data['year'] = year

    data['season'] = get_season(match)

    return data

In [128]:
test_func = parse_match(obj, key1)

1-2


In [129]:
print(test_func)

{'player_left': 'Bunny', 'player_left_race': 't', 'player_right': 'MyuNgSiK', 'player_right_race': 'p', 'score': '1-2', 'winner': 'MyuNgSiK', 'loser': 'MyuNgSiK', 'match_no': '4', 'match_date': 'April 8, 2016', 'year': '2016', 'season': '1'}


In [143]:
gsl_data = list()
for match in list(obj['query']['results'].keys()):
    gsl_data.append(parse_match(obj, match))

In [133]:
list(obj['query']['results'].keys())[0]

'2016 Global StarCraft II League Season 1/Code S#Match Bunny vs MyuNgSiK at April 8, 2016 - 18:30 in Match 4'

In [136]:
parse_match(obj, list(obj['query']['results'].keys())[0])

{'player_left': 'Bunny',
 'player_left_race': 't',
 'player_right': 'MyuNgSiK',
 'player_right_race': 'p',
 'score': '1-2',
 'winner': 'MyuNgSiK',
 'loser': 'MyuNgSiK',
 'match_no': '4',
 'match_date': 'April 8, 2016',
 'year': '2016',
 'season': '1'}

In [144]:
df = pd.DataFrame(gsl_data)
df.head()

,player_left,player_left_race,player_right,player_right_race,score,winner,loser,match_no,match_date,year,season
0,Bunny,t,MyuNgSiK,p,1-2,MyuNgSiK,Bunny,4,"April 8, 2016",2016,1
1,Bunny,t,Symbol,z,2-0,Bunny,Symbol,4,"February 24, 2016",2016,1
2,Classic,p,SpeeD,t,1-2,SpeeD,Classic,5,"March 4, 2016",2016,1
3,Creator,p,Classic,p,1-2,Classic,Creator,2,"March 4, 2016",2016,1
4,Cure,t,TY,t,0-4,TY,Cure,1,"April 20, 2016",2016,1


In [147]:
df.dtypes

player_left          object
player_left_race     object
player_right         object
player_right_race    object
score                object
winner               object
loser                object
match_no             object
match_date           object
year                 object
season               object
dtype: object

In [149]:
df.match_date = pd.to_datetime(df.match_date)


In [151]:
df = df.sort_values(by=['match_date'])

In [152]:
df.head()

,player_left,player_left_race,player_right,player_right_race,score,winner,loser,match_no,match_date,year,season
33,Rogue,z,Journey,t,0-2,Journey,Rogue,1,2016-02-17,2016,1
50,TaeJa,t,Zest,p,0-2,Zest,TaeJa,2,2016-02-17,2016,1
34,Rogue,z,TaeJa,t,0-2,TaeJa,Rogue,4,2016-02-17,2016,1
22,Journey,t,TaeJa,t,1-2,TaeJa,Journey,5,2016-02-17,2016,1
23,Journey,t,Zest,p,1-2,Zest,Journey,3,2016-02-17,2016,1


In [153]:
df.to_excel('prelim_results.xlsx')

In [ ]:
# Check if herO has page
# check URL of page
# Label the round ro32/24, ro16, playoffs
# sort by date, then match number
# filter matches 1/2 and then 5 to see if they match
# get counts 
# add column for whether match 1/2 winner won match 5